In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy
import re
import string
from collections import Counter

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [673]:
imdb = pd.read_csv('..\..\imdb_labelled.txt', sep='\t|', engine='python', header=None)
imdb.columns = ['desc', 'score']

C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\io\parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\io\parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [ ]:
with open('..\..\imdb_labelled.txt') as imdb_file: # almost always use the 'with' statement 
    text = imdb_file.readlines()    # when opening; dont have to use .close()
    print("This file is {} lines long\n".format(len(text)))
    for line in text:
        print(line)

## decide what features to add, or what words to add to features

In [ ]:
imdb['categ'] = imdb['score'].apply(lambda x:'pos' if x==1 else 'neg')

In [ ]:
imdb_neg = imdb.copy()[imdb['score'] == 0]
imdb_pos = imdb.copy()[imdb['score'] == 1]


In [ ]:
# imdb_pos['score'] = 'pos'
# imdb_neg['score'] = 'neg'

In [ ]:
pos_words = imdb_pos['desc'].tolist()

In [ ]:
neg_words = imdb_neg['desc'].tolist()

In [ ]:
all_words = imdb['desc'].tolist()

In [ ]:
def features(sentence):
    words = sentence.lower().split()
    return dict(('%s' % w, True) for w in words)

In [ ]:
pos_featuresets = list(map(features, pos_words))

In [ ]:
neg_featuresets = list(map(features, neg_words))

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
text_mnb = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB())
                    ])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imdb.desc, imdb.score)

In [ ]:
text_mnb.fit(X_train, y_train)

In [ ]:
predicted = text_mnb.predict(X_test)

In [ ]:
np.mean(predicted == y_test)

In [ ]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

In [ ]:
metrics.confusion_matrix(y_test, predicted)

In [ ]:
count_vect = CountVectorizer()

In [ ]:
X_train_counts = count_vect.fit_transform(X_train)

In [ ]:
count_vect.vocabulary_

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [ ]:
twenty_train = fetch_20newsgroups(subset='train',
...     categories=categories, shuffle=True, random_state=42)

In [ ]:
twenty_train.target_names

In [ ]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

In [ ]:
twenty_train.target_names[twenty_train.target[0]]

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vect = CountVectorizer()

In [ ]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [ ]:
X_train_counts

In [ ]:
count_vect.vocabulary_.get(u'algorithm')

In [ ]:
count_vect.vocabulary_

In [ ]:
# from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# tf
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [ ]:
X_train_tf = tf_transformer.transform(X_train_counts)

In [ ]:
# tf-idf; Term Frequency times Inverse Document Frequency
tfidf_transformer = TfidfTransformer()

In [ ]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
# from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [ ]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']

In [ ]:
X_new_counts = count_vect.transform(docs_new)

In [ ]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [ ]:
predicted = clf.predict(X_new_tfidf)

In [ ]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' %(doc, twenty_train.target_names[category]))

In [ ]:
# make vectorizer => transformer => classifier easier to work with...
# Pipeline class
# from sklearn.pipeline import Pipeline

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB()),
])

In [ ]:
text_clf.fit(twenty_train.data, twenty_train.target)

In [ ]:
twenty_test = fetch_20newsgroups(subset='test',
                          categories=categories, shuffle=True, random_state=42)

In [ ]:
docs_test = twenty_test.data

In [ ]:
predicted = text_clf.predict(docs_test)

In [ ]:
np.mean(predicted == twenty_test.target)

In [ ]:
# support vector machine (SVM), regarded as one of best text clf algorithms 
# (but slow)
from sklearn.linear_model import SGDClassifier

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                         alpha=1e-3, random_state=42,
                                         max_iter=5, tol=None)),
])

In [ ]:
text_clf.fit(twenty_train.data, twenty_train.target)

In [ ]:
predicted = text_clf.predict(docs_test)

In [ ]:
np.mean(predicted==twenty_test.target)

In [ ]:
# from sklearn import metrics

In [ ]:
print(metrics.classification_report(twenty_test.target, predicted,
                                   target_names=twenty_test.target_names))

In [ ]:
metrics.confusion_matrix(twenty_test.target, predicted)

In [ ]:
line_length = []
for l in a:
    line_length.append(len(re.split(', | |-|:|\\n|\\t  ', l)))


In [ ]:
l = []
for i, row in imdb.iterrows():
    if 'good' in row['desc']:
        l.append(1)
    else:
        l.append(0)
        

In [ ]:
imdb['good'] = l

In [ ]:
for i, row in imdb.iterrows():
    print (row)

In [ ]:
string.punctuation

In [ ]:

# words = re.split(r'\W+', a[0])
words = a[788].split()
table = str.maketrans('', '', string.punctuation)
stripped = [w.lower().translate(table) for w in words]
stripped

In [ ]:

for i in a.index:
    print(re.split(', | |-', a[i]))

In [ ]:

count_word = Counter()
punct = ',.!?:;\'"-()*&^%$#@/\\'
for l in a:
    count_word.update(x.strip(punct).lower() for x in re.split(', | |-|:|\\n|\\t', l))

In [ ]:
re.split(', | |-', a[0])[4].rstrip(punct).lower()

In [ ]:

re.split(', | |-|\\n|\\t', '\t1\nthe')

In [ ]:
re.split(', | |-', 'slow-moving')

In [ ]:
count_word

In [ ]:
import operator
sorted_count = sorted(count_word.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
sorted_count

# not needed

In [ ]:
# create dictionary with frequency count
# use regex and Counter method
count_word1 = {}
punct = ',.!?\'"-*&^%$#@'
for i in a:
    for word in i:
        if word not in count_word:
            count_word[word] = 1
        else:
            count_word[word] += 1

In [ ]:
imdb_neg

In [ ]:
imdb_neg.desc[0]

In [ ]:
re.split(', | |-', imdb_neg.desc[0].lower())[:-2]

In [ ]:
b = imdb_neg.desc.apply(lambda x: re.split(', | |-', x.lower())[:-2])
b

In [ ]:
# imdb_neg['pract'] = b